In [3]:
import pandas as pd
df = pd.read_csv('directors-movies.csv')

movie_overviews = df["overview"].tolist()

In [4]:
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv(".env")

embeddings = OpenAIEmbeddings()

embeddings_list = []

for overview in movie_overviews:
  embeddings_list.append(embeddings.embed_query(overview))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
import numpy as np

embeddings_list = np.array(embeddings_list).astype("float32")

In [6]:
import faiss

index = faiss.IndexFlatL2(1536)
index.add(embeddings_list)

In [7]:
faiss.write_index(index, "embeddings.index")

In [8]:
index = faiss.read_index("embeddings.index")

In [9]:
test_sample = embeddings.embed_query("When teenager Elle's first kiss leads to a forbidden romance with the hottest boy in high school, she risks her relationship with her best friend.")

test_sample = np.array([test_sample]).astype("float32")

In [10]:
k = 5
distances, indices = index.search(test_sample, k)

In [18]:
recs = df.iloc[indices[0]]
recs_dict = recs.to_dict(orient='records')
print(recs_dict)

[{'id': 987923, 'title': 'Young Adult', 'original_language': 'en', 'overview': 'On the last day of a summer camp for persons with disabilities, a teenage girl fights to connect the boy that might be the love of her life.', 'poster_path': '/A8g8rTSpdDKvKC1Y5zgf2NnwaD2.jpg', 'genre_ids': '[18]', 'popularity': 0.846, 'release_date': '2017-01-01', 'vote_average': 0.0, 'vote_count': 0}, {'id': 634022, 'title': 'Waking Hour', 'original_language': 'en', 'overview': 'A young trans woman is pursued by a potential lover at a party. As she balances concerns about her safety and her desire for intimacy, she must figure out what she wants before she can go for it.', 'poster_path': '/jznXxLXZ8ilNiAXHjOBO5vVrYyN.jpg', 'genre_ids': '[]', 'popularity': 1.349, 'release_date': '2019-03-04', 'vote_average': 0.0, 'vote_count': 0}, {'id': 177354, 'title': 'Stud Life', 'original_language': 'en', 'overview': "Stud lesbian JJ works with her gay best friend Seb as wedding photographers. When JJ falls in love wi

prepend "https://image.tmdb.org/t/p/w1280" to 'poster_path' to get a link to the movie's image; prepend "https://www.themoviedb.org/movie/" to the id to get a link to the movie itself